# **Programming Assignment 2 - Object Detection + Semantic Segmentation**

#### **Professor**: Dário Oliveira  
#### **Monitora**: Lívia Meinhardt


O objetivo deste trabalho é construir um pipeline de visão computacional que primeiro **detecta** objetos em uma imagem e, em seguida, realiza a **segmentação semântica** em cada objeto detectado. Ou seja, uma segmentação de instância em duas etapas. 

Vocês irão construir e conectar dois modelos distintos:
1.  Um **detector de objetos** (YOLO) para encontrar a localização dos objetos.
2.  Um **segmentador semântico** (U-Net ou outro) para classificar os pixels dentro de cada objeto localizado.

### **Instruções:**

1. **Criação de um Dataset**:  
Vocês usarão o dataset **[PASCAL VOC 2012](https://www.kaggle.com/datasets/gopalbhattrai/pascal-voc-2012-dataset)**. Este conjunto de dados é ideal porque fornece anotações tanto para **caixas delimitadoras** (*bounding boxes*), para a tarefa de detecção, quanto para **máscaras de segmentação** a nível de pixel. 


2. **Implementação e Treinamento da YOLO:**
Sua primeira tarefa é fazer a implementação da YOLOv3, vista em aula. Treine um modelo que recebe uma imagem como entrada e retorna uma lista de predições, onde cada predição contém `(caixa_delimitadora, classe, score_de_confiança)`. Meça o desempenho do seu detector usando a métrica **Mean Average Precision (mAP)**.


3. **Treinar o Segmentador Semântico:**
Sua segunda tarefa é treinar um modelo **U-Net** ou outro de sua preferência para realizar a segmentação. O ponto crucial é que vocês não irão treinar a U-Net com imagens inteiras. Em vez disso, vocês a treinarão com **recortes de imagem (*patches*)** gerados a partir das caixas delimitadoras de referência (*ground-truth*) do dataset. Meça o desempenho do seu segmentador usando a métrica **Average Precision (AP).**

4. **Construir o Pipeline de Inferência:**
    Agora, conecte seus dois modelos treinados. Esta parte envolve escrever um script que executa a tarefa completa de ponta a ponta.

    1.  **Detectar**: Use uma nova imagem de teste e passe-a pelo seu modelo **YOLOv3** treinado para obter uma lista de caixas delimitadoras preditas.
    2.  **Recortar**: Para cada predição de alta confiança do YOLO, **recorte o *patch* da imagem** definido pela caixa delimitadora.
    3.  **Segmentar**: Passe cada *patch* recortado pelo seu modelo **U-Net** treinado para obter uma máscara de segmentação para aquele objeto específico.
    4.  **Combinar**: Crie uma imagem em branco (preta) do mesmo tamanho da imagem original. "Costure" cada máscara gerada de volta nesta imagem em branco, na sua localização original da caixa delimitadora.
    5.  **Visualizar**: Sobreponha a máscara final combinada na imagem original para criar uma visualização final mostrando todos os objetos detectados e segmentados.


5. **Compare com um método *end-to-end:***
Por fim, faça *fine-tuning* do [**Mask R-CNN**](https://docs.pytorch.org/vision/main/models/mask_rcnn.html), um um modelo de segmentação de instância de ponta a ponta (*end-to-end*). Compare o desempenho com o seu pipeline de dois estágios e discuta as diferenças. 


### **Entrega:**

Você deve enviar:

1.  Um **Jupyter Notebook** contendo todo o seu código.
2.  Os **pesos treinados** tanto do seu detector YOLOv3 quanto do seu segmentador U-Net.
3.  Um **relatório ou apresentação** que discuta os desafios e resultados dos seus experimentos. 

## Preparando o dataset pra detecção

In [ ]:
import kagglehub
gopalbhattrai_pascal_voc_2012_dataset_path = kagglehub.dataset_download('gopalbhattrai/pascal-voc-2012-dataset')

print('Data source import complete.')

In [ ]:
gopalbhattrai_pascal_voc_2012_dataset_path

In [ ]:
import torch
from torch import nn
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

from torchvision.transforms import ToTensor, Lambda
from torchvision.io import decode_image
import torchvision.transforms as T
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights
from torchvision.models.segmentation.deeplabv3 import DeepLabHead, FCNHead

#from torchtune.datasets import ConcatDataset

from sklearn.metrics import confusion_matrix


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

from tqdm import tqdm

import os
import glob

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"using device '{device}'")

In [ ]:
import os
import random
from PIL import Image
import shutil
from ultralytics import YOLO

# Define dataset root
original_dataset_path = 'data'
yolo_dataset_path = 'yolo_dataset'

In [6]:
yolo_dirs = [
    os.path.join(yolo_dataset_path, 'images', 'train'),
    os.path.join(yolo_dataset_path, 'images', 'val'),
    os.path.join(yolo_dataset_path, 'labels', 'train'),
    os.path.join(yolo_dataset_path, 'labels', 'val')
]

for yolo_dir in yolo_dirs:
    os.makedirs(yolo_dir, exist_ok=True)

jpeg_images_dir = os.path.join(original_dataset_path, 'VOC2012_train_val', 'JPEGImages')
annotations_dir = os.path.join(original_dataset_path, 'VOC2012_train_val', 'Annotations')
if not os.path.exists(jpeg_images_dir) or not os.path.exists(annotations_dir):
    raise FileNotFoundError(f"The directory {jpeg_images_dir} or {annotations_dir} does not exist. Please verify the dataset path.")
image_filenames = os.listdir(jpeg_images_dir)
image_ids = [os.path.splitext(filename)[0] for filename in image_filenames if filename.endswith('.jpg')]

random.seed(42)
random.shuffle(image_ids)
split_index = int(0.8 * len(image_ids)) #Spliting the dataset 80% for training, 20% for validation
train_ids = image_ids[:split_index] #taking the first 80% pictures
val_ids = image_ids[split_index:]

FileNotFoundError: The directory /home/pedro/Modelos/Faculdade/DL/Assignment 2/data/VOC2012_train_val/JPEGImages or /home/pedro/Modelos/Faculdade/DL/Assignment 2/data/VOC2012_train_val/Annotations does not exist. Please verify the dataset path.

In [ ]:
import xml.etree.ElementTree as ET 
#this fucntion converts PASCAL_VOC annotations to YOLO format
def create_yolo_annotation(xml_file_path, yolo_label_path, label_dict):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    annotations = [] #list that will store the converted YOLO annotations.

    img_width = int(root.find('size/width').text)
    img_height = int(root.find('size/height').text)

    for obj in root.findall('object'):
        label = obj.find('name').text
        if label not in label_dict:
            continue
        label_idx = label_dict[label]
        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)

        # this is YOLOv8 annotation format: label x_center y_center width height (normalized)
        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        annotations.append(f"{label_idx} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    #annotations to the label file
    with open(yolo_label_path, 'w') as f:
        f.write("\n".join(annotations))

In [ ]:
label_dict = {
    'aeroplane': 0, 'bicycle': 1, 'bird': 2, 'boat': 3, 'bottle': 4,
    'bus': 5, 'car': 6, 'cat': 7, 'chair': 8, 'cow': 9,
    'diningtable': 10, 'dog': 11, 'horse': 12, 'motorbike': 13, 'person': 14,
    'pottedplant': 15, 'sheep': 16, 'sofa': 17, 'train': 18, 'tvmonitor': 19
}

for image_set, ids in [('train', train_ids), ('val', val_ids)]:
    for img_id in ids:
        img_src_path = os.path.join(jpeg_images_dir, f'{img_id}.jpg')
        label_dst_path = os.path.join(yolo_dataset_path, 'labels', image_set, f'{img_id}.txt')

        # Create the YOLO annotation file
        xml_file_path = os.path.join(annotations_dir, f'{img_id}.xml')
        if not os.path.exists(xml_file_path):
            print(f"Warning: Annotation {xml_file_path} not found, skipping.")
            continue
        create_yolo_annotation(xml_file_path, label_dst_path, label_dict)

        # Copy the image to the new YOLO dataset structure
        img_dst_path = os.path.join(yolo_dataset_path, 'images', image_set, f'{img_id}.jpg')
        shutil.copy(img_src_path, img_dst_path)

### Testando na YOLOv8

In [ ]:
yaml_content = f"""
train: images/train
val: images/val

nc: {len(label_dict)}
names: {list(label_dict.keys())}
"""

with open(os.path.join(yolo_dataset_path, 'data.yaml'), 'w') as f:
    f.write(yaml_content)

In [ ]:
model = YOLO('yolov8n.pt')  # Use the YOLOv8 nano model

# Train the model
model.train(
    data=os.path.join(yolo_dataset_path, 'data.yaml'),  # Path to dataset config file
    epochs=2,  # Number of epochs
    imgsz=640,  # Image size
    batch=16,  # Batch size
    name='yolov8_test'  # Experiment name
)

In [ ]:
results = model.predict(source=os.path.join(yolo_dataset_path, 'images/val/'), save=True)  # Run inference on validation set

# Display predictions
for result in results:
    result.show()
    break

In [ ]:
max_conf = 0
best_result = None

for result in results:    
        if (result.boxes.conf.mean().item() > max_conf) and (result.boxes.conf.mean().item() < 0.98) and (result.boxes.conf.shape[0] > 2):
            max_conf = result.boxes.conf.mean().item()
            best_result = result

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

img = best_result.orig_img
boxes = best_result.boxes.xyxy.cpu().numpy()  # xyxy format
classe = best_result.boxes.cls
confianca = best_result.boxes.conf

fig, ax = plt.subplots(1)
ax.imshow(img)

for i, box in enumerate(boxes):
    x1, y1, x2, y2 = box[:4]
    rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    # Add class and confidence
    class_idx = int(classe[i])
    class_name = best_result.names[class_idx] if hasattr(best_result, 'names') else str(class_idx)
    conf = float(confianca[i])
    ax.text(x1, y1 - 5, f'{class_name}: {conf:.2f}', color='red', fontsize=10, backgroundcolor='white')

plt.show()

# Implementando YOLO V3

In [11]:
class Trainer():
    def __init__(self, model, train_loader, val_loader, criterion, optimizer, device):
        self.device = device
        self.model = model.to(device)

        self.train_loader = train_loader
        self.val_loader = val_loader
        
        self.criterion = criterion
        self.optimizer = optimizer
        
        self.history = {
            "train_loss": [],
            "val_loss": []
        }

    def train_one_epoch(self):
        self.model.train()
        running_loss = 0.0

        progress_bar = tqdm(self.train_loader, desc="Training")
        for X, y in progress_bar:
            X, y = X.to(self.device), y.to(self.device)

            pred = self.model(X)
            loss = self.criterion(pred, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item() * X.size(0)
            
            progress_bar.set_postfix(loss=loss.item())

        epoch_loss = running_loss / len(self.train_loader.dataset)
        return epoch_loss

    def validate_one_epoch(self):
        self.model.eval()
        running_loss = 0.0

        with torch.no_grad():
            progress_bar = tqdm(self.val_loader, desc="Validating")
            for X, y in progress_bar:
                X, y = X.to(self.device), y.to(self.device)

                pred = self.model(X)
                loss = self.criterion(pred, y)

                running_loss += loss.item() * X.size(0)

                progress_bar.set_postfix(loss=loss.item())

        epoch_loss = running_loss / len(self.val_loader.dataset)
        return epoch_loss

    def fit(self, num_epochs):
        for epoch in range(num_epochs):
            train_loss = self.train_one_epoch()
            val_loss = self.validate_one_epoch()

            self.history["train_loss"].append(train_loss)
            self.history["val_loss"].append(val_loss)

        return self.history

In [12]:
from dataclasses import dataclass, asdict, field
from typing import Literal, Union

@dataclass
class ConvBlockSpec:
    out_channels: int
    kernel_size: int
    stride: int
    padding: int = field(init=False)

    def __post_init__(self):
        self.padding = 1 if self.kernel_size == 3 else 0

@dataclass
class ResidualBlockSpec:
    num_repeats: int
    channels: int

@dataclass
class ScaleBlockSpec:
    from_layer: int = -1

@dataclass
class UpsampleBlockSpec:
    from_layer: int = -1

config = [
    #===DARKNET-53===
    ConvBlockSpec(32, 3, 1),
    ConvBlockSpec(64, 3, 2),
    ResidualBlockSpec(1, 64),
    
    ConvBlockSpec(128, 3, 2),
    ResidualBlockSpec(2, 128),
    
    ConvBlockSpec(256, 3, 2),
    ResidualBlockSpec(8, 256),
    
    ConvBlockSpec(512, 3, 2),
    ResidualBlockSpec(8, 512),

    ConvBlockSpec(1024, 3, 2),
    ResidualBlockSpec(4, 1024),
    #===DARKNET-53===

    ConvBlockSpec(512, 1, 1),
    ConvBlockSpec(1024, 1, 1),
    ScaleBlockSpec(),
    
    ConvBlockSpec(256, 1, 1),
    UpsampleBlockSpec(),
    
    ConvBlockSpec(256, 1, 1),
    ConvBlockSpec(512, 1, 1),
    ScaleBlockSpec(),

    ConvBlockSpec(128, 1, 1),
    UpsampleBlockSpec(),

    ConvBlockSpec(128, 1, 1),
    ConvBlockSpec(256, 1, 1),
    ScaleBlockSpec(),
]

In [92]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, bn_act=True, **kwargs):
        super().__init__()

        use_bias = not bn_act
        layers = []

        layers.append(
            nn.Conv2d(in_channels, out_channels, bias=use_bias, **kwargs)
        )

        if bn_act:
            layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.LeakyReLU(0.1, inplace=True))

        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)


class ResidualBlock(nn.Module):
    def __init__(self, channels, use_residuals=True, num_repeats=1):
        super().__init__()

        self.use_residuals = use_residuals
        self.num_repeats = num_repeats

        layers = nn.ModuleList()

        for _ in range(num_repeats):
            layers.append(
                nn.Sequential(
                    CNNBlock(channels, channels//2, kernel_size=1),
                    CNNBlock(channels//2, channels, kernel_size=3, padding=1),
                )
            )

        self.block = nn.Sequential(*layers)

    def forward(self, x):
        for layer in self.block:
            if self.use_residuals:
                print(x.shape)
                x = x + layer(x)
            else:
                x = layer(x)
        return x


class ScaleBlock(nn.Module):
    def __init__(self, in_channels, num_classes, num_anchors=3):
        super().__init__()

        self.num_classes = num_classes
        self.num_anchors = num_anchors

        self.pred = nn.Sequential(
            CNNBlock(in_channels, 2*in_channels, kernel_size=3, padding=1),
            CNNBlock(
                2*in_channels,
                self.num_anchors *(num_classes+5),
                bn_act=False,
                kernel_size=1
            ),
        )

        # [p, cx, cy, w, h, Ic1, ..., Icn] * num_anchors
        
    def forward(self, x):
        pred = self.pred(x)

        # N, C, W, H
        N, _, S, _ = pred.shape

        pred = pred.reshape(N, self.num_anchors, self.num_classes+5, S, S)
        pred = pred.permute(0, 1, 3, 4, 2)

        return pred


class YOLOv3(nn.Module):
    def __init__(self, config, num_anchors=3, in_channels=3, num_classes=20):
        super().__init__()
        self.config = config
        self.in_channels = 3
        self.num_classes = 20
        self.num_anchors = num_anchors

        self.layers = self.gen_layers()

    def gen_layers(self):
        layers = nn.ModuleList()
        in_channels = self.in_channels

        for layer in self.config:
            if isinstance(layer, ConvBlockSpec):
                layers.append(CNNBlock(in_channels, **asdict(layer)))
                in_channels = layer.out_channels
            elif isinstance(layer, ResidualBlockSpec):
                layers.append(ResidualBlock(**asdict(layer)))
            elif isinstance(layer, ScaleBlockSpec):
                layers += [
                    ResidualBlock(in_channels, use_residuals=False, num_repeats=1),
                    CNNBlock(in_channels, in_channels//2, kernel_size=1),
                    ScaleBlock(in_channels//2, self.num_classes, num_anchors=self.num_anchors)
                ]
                in_channels = in_channels//2
                
            elif isinstance(layer, UpsampleBlockSpec):
                layers.append(nn.Upsample(scale_factor=2))
                in_channels = in_channels*3

        return layers

    def forward(self, x):
        preds = []
        route_connections = []

        #print(self.layers)

        for layer in self.layers:
            # print(isinstance(layer, CNNBlock))
            # print(isinstance(layer, ResidualBlock))
            # print(isinstance(layer, ScaleBlock))
            # print()

            if isinstance(layer, ScaleBlock):
                print("scale")
                preds.append(layer(x))
                continue
            
            x = layer(x)
            
            if isinstance(layer, ResidualBlock) and layer.num_repeats == 8:
                route_connections.append(x)

            if isinstance(layer, nn.Upsample):
                print("concat")
                x = torch.cat([x, route_connections.pop()], dim=1)

        return preds

yolo = YOLOv3(config, num_anchors=2)
x = torch.randn((10, 3, 416, 416))
out = yolo(x)

torch.Size([10, 64, 208, 208])
torch.Size([10, 128, 104, 104])
torch.Size([10, 128, 104, 104])
torch.Size([10, 256, 52, 52])
torch.Size([10, 256, 52, 52])
torch.Size([10, 256, 52, 52])
torch.Size([10, 256, 52, 52])
torch.Size([10, 256, 52, 52])
torch.Size([10, 256, 52, 52])
torch.Size([10, 256, 52, 52])
torch.Size([10, 256, 52, 52])
torch.Size([10, 512, 26, 26])
torch.Size([10, 512, 26, 26])
torch.Size([10, 512, 26, 26])
torch.Size([10, 512, 26, 26])
torch.Size([10, 512, 26, 26])
torch.Size([10, 512, 26, 26])
torch.Size([10, 512, 26, 26])
torch.Size([10, 512, 26, 26])
torch.Size([10, 1024, 13, 13])
torch.Size([10, 1024, 13, 13])
torch.Size([10, 1024, 13, 13])
torch.Size([10, 1024, 13, 13])
scale
concat
scale
concat
scale


In [99]:
counter = 0

for img in out[1]:
    print(img.shape)
    counter += 1

print(counter)
    #print(out[i].shape)

torch.Size([2, 26, 26, 25])
torch.Size([2, 26, 26, 25])
torch.Size([2, 26, 26, 25])
torch.Size([2, 26, 26, 25])
torch.Size([2, 26, 26, 25])
torch.Size([2, 26, 26, 25])
torch.Size([2, 26, 26, 25])
torch.Size([2, 26, 26, 25])
torch.Size([2, 26, 26, 25])
torch.Size([2, 26, 26, 25])
10


In [ ]:
import torch
import numpy as np
import cv2
import torchvision

# --- PARÂMETROS E DADOS SIMULADOS ---
# (Substitua estes pelos seus dados reais)

# 1. Parâmetros do modelo e pós-processamento
CONF_THRESHOLD = 0.5  # Confiança mínima para considerar uma detecção
NMS_THRESHOLD = 0.4   # Limiar de IoU para a Supressão Não Máxima (NMS)
IMG_SIZE = 416        # Tamanho da imagem que o modelo espera (ex: 416x416)
NUM_CLASSES = 20      # Número de classes (ex: 20 para PASCAL VOC)

# 2. Nomes das classes (exemplo com 20 classes do PASCAL VOC)
CLASS_NAMES = [
    "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat",
    "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person",
    "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]

# 3. Âncoras usadas pela YOLOv3 para a escala de 52x52 (objetos pequenos)
# (Estes valores são relativos ao tamanho da imagem, ex: 416)
ANCHORS_52x52 = [(10, 13), (16, 30), (33, 23)]

# 4. Criar uma imagem preta de exemplo
# Na prática, você carregaria sua imagem com: cv2.imread("sua_imagem.jpg")
dummy_image = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)

# 5. Criar uma saída de modelo YOLO simulada (torch.Size([1, 3, 13, 13, 25]))
# O formato é [batch_size, num_anchors, grid_h, grid_w, 5 + num_classes]
# Usaremos uma grade 13x13 para simplificar a visualização
GRID_SIZE = 13
dummy_output = torch.zeros(1, 3, GRID_SIZE, GRID_SIZE, 5 + NUM_CLASSES)

# ---- Vamos "forçar" uma detecção para ter algo para plotar ----
# Classe 6: "car" | Âncora 1 | Célula da grade (4, 4)
dummy_output[0, 1, 4, 4, 0] = 0.8   # tx (centro x) -> próximo ao centro da célula
dummy_output[0, 1, 4, 4, 1] = 0.8   # ty (centro y) -> próximo ao centro da célula
dummy_output[0, 1, 4, 4, 2] = 0.5   # tw (largura) -> um pouco maior que a âncora
dummy_output[0, 1, 4, 4, 3] = 0.5   # th (altura) -> um pouco maior que a âncora
dummy_output[0, 1, 4, 4, 4] = 0.98  # Confiança de que há um objeto (objectness)
dummy_output[0, 1, 4, 4, 5 + 6] = 0.99 # Probabilidade da classe "car"


# --- FUNÇÕES DE PROCESSAMENTO E PLOTAGEM ---

def process_yolo_output(yolo_output, conf_threshold, nms_threshold, anchors):
    """
    Processa a saída bruta da YOLO para obter as caixas finais.
    """
    predictions = yolo_output.clone()
    
    # Obter dimensões
    batch_size, num_anchors, grid_h, grid_w, num_attrs = predictions.shape
    
    # 1. Transformar as coordenadas da saída
    # A saída da YOLO precisa ser convertida para coordenadas de imagem reais.
    
    # Criar uma grade de coordenadas (ex: [0,1,2...12])
    grid_x = torch.arange(grid_w).repeat(grid_h, 1).view([1, 1, grid_h, grid_w]).float()
    grid_y = torch.arange(grid_h).repeat(grid_w, 1).t().view([1, 1, grid_h, grid_w]).float()

    # Converter âncoras para tensores
    tensor_anchors = torch.tensor(anchors).float().view(1, num_anchors, 1, 1, 2)
    
    # Aplicar as fórmulas de transformação da YOLO
    # Ativar coordenadas de centro (tx, ty) e confiança (obj) com sigmoide
    predictions[..., 0:2] = torch.sigmoid(predictions[..., 0:2])  # Centro x, y
    predictions[..., 4] = torch.sigmoid(predictions[..., 4])      # Confiança (objectness)
    
    # Adicionar o offset da grade para obter a posição na imagem
    predictions[..., 0] += grid_x
    predictions[..., 1] += grid_y
    
    # Aplicar a fórmula da exponencial para largura e altura
    predictions[..., 2:4] = torch.exp(predictions[..., 2:4]) * tensor_anchors
    
    # Ativar as probabilidades de classe com sigmoide
    predictions[..., 5:] = torch.sigmoid(predictions[..., 5:])
    
    # Escalonar as coordenadas para o tamanho real da imagem (ex: 416x416)
    stride = IMG_SIZE / grid_w
    predictions[..., :4] *= stride
    
    # 2. Filtrar as previsões
    # Transformar de [batch, anchors, grid, grid, attrs] para uma lista de caixas
    predictions = predictions.view(batch_size, -1, num_attrs) # [-1] achata as dimensões de âncora e grade
    
    output_boxes = []
    for batch_i in range(batch_size):
        preds = predictions[batch_i]
        
        # Filtrar por confiança de objeto
        mask = preds[:, 4] > conf_threshold
        preds = preds[mask]
        
        if not preds.size(0):
            continue
            
        # Obter a classe com maior probabilidade
        class_scores, class_ids = torch.max(preds[:, 5:], 1)
        
        # Juntar tudo em um formato [x1, y1, x2, y2, obj_conf, class_score, class_id]
        # Converter (centro_x, centro_y, w, h) para (x1, y1, x2, y2)
        box_center_x, box_center_y, box_w, box_h = preds[:, 0], preds[:, 1], preds[:, 2], preds[:, 3]
        x1 = box_center_x - box_w / 2
        y1 = box_center_y - box_h / 2
        x2 = box_center_x + box_w / 2
        y2 = box_center_y + box_h / 2
        
        # 3. Aplicar Supressão Não Máxima (NMS)
        # NMS remove caixas redundantes para o mesmo objeto
        boxes_for_nms = torch.stack([x1, y1, x2, y2], dim=1)
        scores = preds[:, 4] * class_scores # Usar a confiança do objeto e da classe
        
        indices = torchvision.ops.nms(boxes_for_nms, scores, nms_threshold)
        
        # Selecionar apenas as caixas que sobreviveram ao NMS
        final_boxes = []
        for i in indices:
            final_boxes.append({
                "box": [x1[i].item(), y1[i].item(), x2[i].item(), y2[i].item()],
                "class_id": class_ids[i].item(),
                "score": scores[i].item()
            })
        output_boxes.append(final_boxes)

    return output_boxes[0] # Retornando apenas o resultado do primeiro item do batch

def draw_boxes(image, boxes, class_names):
    """
    Desenha as caixas delimitadoras na imagem.
    """
    vis_image = image.copy()
    
    for box_info in boxes:
        box = box_info['box']
        class_id = box_info['class_id']
        score = box_info['score']
        
        # Coordenadas como inteiros
        x1, y1, x2, y2 = map(int, box)
        
        # Obter nome e cor da classe
        class_name = class_names[class_id]
        color = (255, 255, 255) # Verde para a caixa
        
        # Desenhar a caixa
        cv2.rectangle(vis_image, (x1, y1), (x2, y2), color, 2)
        
        # Preparar o texto (Classe + Confiança)
        label = f"{class_name}: {score:.2f}"
        
        # Desenhar um fundo para o texto
        (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
        cv2.rectangle(vis_image, (x1, y1 - h - 5), (x1 + w, y1), color, -1)
        
        # Escrever o texto
        cv2.putText(vis_image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    return vis_image

# --- EXECUÇÃO PRINCIPAL ---

# 1. Processar a saída simulada da YOLO
final_detections = process_yolo_output(
    yolo_output=dummy_output,
    conf_threshold=CONF_THRESHOLD,
    nms_threshold=NMS_THRESHOLD,
    anchors=ANCHORS_52x52  # Use as âncoras corretas para a escala
)

# 2. Desenhar as caixas na imagem
image_with_boxes = draw_boxes(dummy_image, final_detections, CLASS_NAMES)

# 3. Exibir a imagem
print(f"Detecções encontradas: {len(final_detections)}")
if len(final_detections) > 0:
    print("Detalhes da primeira detecção:", final_detections[0])

cv2.imshow("YOLO Output Visualization", image_with_boxes)
cv2.waitKey(0)  # Espera uma tecla ser pressionada para fechar
cv2.destroyAllWindows()

Detecções encontradas: 1
Detalhes da primeira detecção: {'box': [-271.99346923828125, -641.3070068359375, 572.1517944335938, 941.46533203125], 'class_id': 6, 'score': 0.5301257967948914}


In [55]:
len(out[1][0,0,0,0,:])

25

In [ ]:
res = ResidualBlock(channels=32)
x = torch.randn((2, 32, 416, 416))
res(x).shape